In [80]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.model_selection import train_test_split
import os

In [81]:
DATA_BASE_PATH = 'DATA'
GENERATOR = '1'
RESULTS_BASE_PATH = 'RESULTS'

NODES = [(5, 8), (8, 5), (5, 12), (8, 12)]

# FAILUER_TIME = 16000

In [82]:
def load_sequence(file):
    Y = np.load(os.path.join(DATA_BASE_PATH, GENERATOR, f"{file}.npy"))
    Y = Y.reshape((-1))

    return Y

def split_sequence(sequence, n_steps_in, n_steps_out):
    X = [sequence[i:i+n_steps_in]
         for i in range(len(sequence) - (n_steps_in + n_steps_out) + 1)]
    y = [sequence[i:i+n_steps_out]
         for i in range(n_steps_in, len(sequence) - n_steps_out + 1)]

    return np.array(X).reshape(len(X), n_steps_in, 1), np.array(y)

def define_model(n_steps_in, n_steps_out) -> Sequential:
    model = Sequential()
    model.add(LSTM(10, activation='relu', input_shape=(n_steps_in, 1)))
    model.add(Dense(n_steps_out))
    model.compile(optimizer='adam', loss='mse')
    
    return model

def run_experiment(exp, file, n_steps_in, n_steps_out, train_size, epochs=5):
    sequence = load_sequence(file)
    X, y = split_sequence(sequence, n_steps_in, n_steps_out)
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size, random_state=42, shuffle=False)
    model = define_model(n_steps_in, n_steps_out)
    model.fit(X_train, y_train, epochs=epochs)
    y_predicted = model.predict(X_test)

    exp_path = os.path.join(RESULTS_BASE_PATH, exp)

    if not os.path.exists(exp_path):
        os.makedirs(exp_path)

    np.save(os.path.join(exp_path, "true.npy"), y_test)
    np.save(os.path.join(exp_path, "predicted.npy"), y_predicted)

def run_all_experiments(exp, n_steps_in, n_steps_out, train_size, epochs=5):
    for n_from, n_to in NODES:
        run_experiment(f'{exp}___{n_from}to{n_to}', f'Y_{n_from}_{n_to}', n_steps_in, n_steps_out, train_size, epochs)

    

In [83]:
run_all_experiments('LSTM_5_in_1_out', 5, 1, 10000, 15)


Epoch 1/15
313/313 [==============================] - 2s 2ms/step - loss: 122018.8672
Epoch 2/15
313/313 [==============================] - 1s 2ms/step - loss: 25361.8594
Epoch 3/15
313/313 [==============================] - 1s 2ms/step - loss: 17459.2188
Epoch 4/15
313/313 [==============================] - 1s 2ms/step - loss: 14508.3691
Epoch 5/15
313/313 [==============================] - 1s 2ms/step - loss: 13818.0352
Epoch 6/15
313/313 [==============================] - 1s 2ms/step - loss: 13447.1035
Epoch 7/15
313/313 [==============================] - 1s 2ms/step - loss: 13095.3252
Epoch 8/15
313/313 [==============================] - 1s 2ms/step - loss: 12602.3076
Epoch 9/15
313/313 [==============================] - 1s 2ms/step - loss: 14007.6113
Epoch 10/15
313/313 [==============================] - 1s 2ms/step - loss: 14048.9746
Epoch 11/15
313/313 [==============================] - 1s 2ms/step - loss: 14308.9326
Epoch 12/15
313/313 [==============================] - 1s 2ms/